## **Feature Extraction: Texture**

Import libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage import io, filters
from skimage.color import rgb2gray

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Image loading, smoothing, converting to grayscale

In [ ]:
nimage = io.imread("/content/gdrive/My Drive/Colab Notebooks/PatternRecognition/2021Fall/normal.jpg")
cimage = io.imread("/content/gdrive/My Drive/Colab Notebooks/PatternRecognition/2021Fall/cancer.jpg")

nimage = filters.gaussian(nimage, sigma=1.5)
cimage = filters.gaussian(cimage, sigma=1.5)

gray_nimage = rgb2gray(nimage)
gray_cimage = rgb2gray(cimage)

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(gray_nimage, cmap=plt.cm.gray)
plt.title('Normal')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(gray_cimage, cmap=plt.cm.gray)
plt.title('Cancer')
plt.axis('off')
plt.show()

# **Edge**

Compute x, y gradients - Sobel

In [ ]:
from scipy import ndimage as nd

grad_x = nd.convolve(gray_cimage, np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]))
grad_y = nd.convolve(gray_cimage, np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]))

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(grad_x, cmap=plt.cm.gray)
plt.title('Gradient_X')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(grad_y, cmap=plt.cm.gray)
plt.title('Gradient_Y')
plt.axis('off')
plt.show()

Compute magnitude and direction

In [ ]:
mag = np.sqrt(np.power(grad_x,2) + np.power(grad_y,2))
theta = np.arctan2(np.power(grad_y,2), np.power(grad_x,2))

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(mag, cmap=plt.cm.gray)
plt.title('Magnitude')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(theta, cmap=plt.cm.gray)
plt.title('Direction')
plt.axis('off')
plt.show()

Extract edges

In [ ]:
plt.figure(figsize=(5,5))
im = mag > 0.5
plt.imshow(im, cmap=plt.cm.gray)
plt.axis('off')

Use Sobel operator

In [ ]:
from skimage import filters

In [ ]:
sobel_cimage = filters.sobel(gray_cimage)
sobel_nimage = filters.sobel(gray_nimage)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(sobel_cimage, cmap=plt.cm.gray)
plt.title('Sobel Edge : Tumor')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(sobel_nimage, cmap=plt.cm.gray)
plt.title('Sobel Edge : Benign')
plt.axis('off')
plt.show()

Use Canny edge detector

In [ ]:
from skimage import feature

In [ ]:
canny_cimage = feature.canny(gray_cimage, sigma=3)
canny_nimage = feature.canny(gray_nimage, sigma=3)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(canny_cimage, cmap=plt.cm.gray)
plt.title('Canny Edge (sigma=3) : Cancer')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(canny_nimage, cmap=plt.cm.gray)
plt.title('Canny Edge (sigma=3) : Normal')
plt.axis('off')
plt.show()

# **GLCM**

Compute grey level co-occurence matrix

In [ ]:
from skimage.feature import greycomatrix, greycoprops

In [ ]:
level = 32 #  typically 256 for an 8-bit image
ccomatrix = greycomatrix(np.uint8((level-1)*gray_cimage), [1, 10], [0, np.pi/4, np.pi/2], level, symmetric=True, normed=True)
ncomatrix = greycomatrix(np.uint8((level-1)*gray_nimage), [1, 10], [0, np.pi/4, np.pi/2], level, symmetric=True, normed=True)

In [ ]:
print(ccomatrix.shape)

In [ ]:
plt.figure(figsize=(15,3))
id=1
for i in np.arange(0,2):
  for j in np.arange(0,3):
    plt.subplot(1,6,id)
    plt.imshow(ccomatrix[:,:,i,j], cmap=plt.cm.gray)
    plt.axis('off')
    id = id + 1
plt.suptitle('Cancer')
plt.show()

In [ ]:
plt.figure(figsize=(15,3))
id=1
for i in np.arange(0,2):
  for j in np.arange(0,3):
    plt.subplot(1,6,id)
    plt.imshow(ncomatrix[:,:,i,j], cmap=plt.cm.gray)
    plt.axis('off')
    id = id + 1
plt.suptitle('Normal')
plt.show()

Compute features using GLCM

In [ ]:
cglcm_feat = greycoprops(ccomatrix, 'energy')
nglcm_feat = greycoprops(ncomatrix, 'energy')

In [ ]:
print(cglcm_feat.shape)

In [ ]:
cglcm_feat = cglcm_feat.reshape(-1,)
nglcm_feat = nglcm_feat.reshape(-1,)
print('Cancer : ', end='\t')
for t in cglcm_feat:
  print('{:.3f}'.format(t), end='\t')
print()
print('Normal : ', end='\t')
for b in nglcm_feat:
  print('{:.3f}'.format(b), end='\t')


# **Gabor Filters**

In [ ]:
from skimage.filters import gabor, gabor_kernel

Apply a Gabor filter

In [ ]:
filt_real, _ = gabor(gray_cimage, frequency=0.2, theta=np.pi/2)

plt.subplot(1,2,1)
plt.imshow(filt_real)
plt.show()

Visualize a Gabor kernel

In [ ]:
gk = gabor_kernel(0.1, theta=np.pi/2)

plt.imshow(gk.real)
plt.show()

Let's generate Gabor filter banks

In [ ]:
gkernels = []
for frequency in (0.08, 0.1, 0.2):
  for theta in (0, np.pi/4, np.pi/2):
    gk = gabor_kernel(frequency, theta=theta)
    gkernels.append(gk.real)

In [ ]:
[gk.shape for gk in gkernels]

In [ ]:
plt.figure(figsize=(5,5))
id=1
for id, gk in enumerate(gkernels):
  plt.subplot(3,3,id+1)
  plt.imshow(gk.real)
  plt.axis('off')


Apply Gabor filter bank

In [ ]:
from scipy import ndimage as nd

In [ ]:
fig = plt.figure(figsize=(15,7))

for id in np.arange(0, 3):
  plt.subplot(2, 3, id+1)
  cgkimage = nd.convolve(gray_cimage, gkernels[id+3])
  plt.imshow(cgkimage, cmap=plt.cm.gray)
  plt.axis('off')
fig.suptitle('Cancer')
plt.show()

fig = plt.figure(figsize=(15,7))

for id in np.arange(0, 3):
  plt.subplot(2, 3, id+1)
  ngkimage = nd.convolve(gray_nimage, gkernels[id+3])
  plt.imshow(ngkimage, cmap=plt.cm.gray)
  plt.axis('off')
fig.suptitle('Normal')
plt.show()

Compute Gabor features

In [ ]:
from scipy import ndimage as nd

def compute_gabor_features(image, gkernels):
  features = np.zeros((len(gkernels), 3), dtype=np.double)
  for id, gk in enumerate(gkernels):
    gkimage = nd.convolve(image, gk.real)
    features[id, 0] = gkimage.mean()
    features[id, 1] = gkimage.var()
    features[id, 2] = np.sum(np.power(gkimage, 2)) / (gkimage.shape[0] * gkimage.shape[1])
  
  return features

In [ ]:
cgabor_feat = compute_gabor_features(gray_cimage, gkernels)
ngabor_feat = compute_gabor_features(gray_nimage, gkernels)

In [ ]:
cgabor_feat = cgabor_feat.reshape(-1,)
ngabor_feat = ngabor_feat.reshape(-1,)

plt.figure(figsize=(10,5))
plt.plot(np.arange(0, len(cgabor_feat)), cgabor_feat, 'bo:', label="Cancer")
plt.plot(np.arange(0, len(ngabor_feat)), ngabor_feat, 'gd:', label="Normal")
plt.legend(loc="best")
plt.show()

# **LBP**

Compute LBPs

In [ ]:
from skimage.feature import local_binary_pattern

In [ ]:
P = 16
R = 2
METHOD = 'uniform'  # 'default', 'ror', 'var'

clbp = local_binary_pattern(gray_cimage, P, R, METHOD)
nlbp = local_binary_pattern(gray_nimage, P, R, METHOD)

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(clbp, cmap=plt.cm.gray)
plt.title('Cacer')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(nlbp, cmap=plt.cm.gray)
plt.title('Normal')
plt.axis('off')
plt.show()

Compute histograms

In [ ]:
rmax = np.uint8(np.max([clbp.max(), nlbp.max()]))

clbp_hist, _ = np.histogram(clbp.reshape(-1,), normed=True, bins=rmax+1, range=(0, rmax+1))
nlbp_hist, _ = np.histogram(nlbp.reshape(-1,), normed=True, bins=rmax+1, range=(0, rmax+1))

In [ ]:
hists = pd.DataFrame({'Cancer': clbp_hist, 'Normal': nlbp_hist})
hists.plot(kind='bar', figsize=(10,5), legend=True)

Compute multi-scale LBPs

In [ ]:
P = 16; R = 2; METHOD = 'uniform'
clbp = local_binary_pattern(gray_cimage, P, R, METHOD)
nlbp = local_binary_pattern(gray_nimage, P, R, METHOD)

P = 24; R = 3; METHOD = 'uniform'
clbp2 = local_binary_pattern(gray_cimage, P, R, METHOD)
nlbp2 = local_binary_pattern(gray_nimage, P, R, METHOD)

Compute multi-dimensional histograms

In [ ]:
rmax = np.uint8(np.max([clbp.max(), nlbp.max()]))
rmax2 = np.uint8(np.max([clbp2.max(), nlbp2.max()]))

In [ ]:
clbp_mhist, _, _ = np.histogram2d(clbp.reshape(-1,), clbp2.reshape(-1,), normed=True, bins=[rmax+1, rmax2+1], range=[[0,rmax+1], [0,rmax2+1]])
nlbp_mhist, _, _ = np.histogram2d(nlbp.reshape(-1,), nlbp2.reshape(-1,), normed=True, bins=[rmax+1, rmax2+1], range=[[0,rmax+1], [0,rmax2+1]])

Draw multi-dimensional histograms

In [ ]:
x = np.arange(0, rmax+1)
y = np.arange(0, rmax2+1)
xs, ys = np.meshgrid(x, y)

xs = xs.reshape(-1,)
ys = ys.reshape(-1,)
clbp_mhist = clbp_mhist.reshape(-1,)
nlbp_mhist = nlbp_mhist.reshape(-1,)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,2,1, projection='3d')
ax.bar3d(xs, ys, 0, 0.5, 0.5, clbp_mhist, color='violet')
ax.set_xlabel('P=16, R=2')
ax.set_ylabel('P=24, R=3')
ax.set_title('Cancer', fontdict={'fontsize':15})
ax = fig.add_subplot(1,2,2, projection='3d')
ax.bar3d(xs, ys, 0, 0.5, 0.5, nlbp_mhist, color='orange')
ax.set_xlabel('P=16, R=2')
ax.set_ylabel('P=24, R=3')
ax.set_title('Normal', fontdict={'fontsize':15})
plt.show()
